In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

import numpy as np
import pandas as pd
import seaborn as sns
import random
from matplotlib import pyplot as plt
import time

# Adjustment of visibility of Datafreames
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

df = pd.read_csv("./data/diabetes.csv")

def grab_col_names(dataframe, cat_th=10, car_th=20):
    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    # report
    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')         # the number of categorical variables
    print(f'num_cols: {len(num_cols)}')         # the number of numerical variables
    print(f'cat_but_car: {len(cat_but_car)}')   # the number of cardinal variables
    print(f'num_but_cat: {len(num_but_cat)}')   # the number of categorical variables that looks numerical
    return cat_cols, num_cols, cat_but_car


cat_cols, num_cols, cat_but_car = grab_col_names(df)

num_cols_miss = [i for i in num_cols if i != "Pregnancies"]
for i in num_cols_miss:
    df[i] = df.apply(lambda x: np.nan if x[i] == 0 else x[i], axis=1)

df_fill = df.apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0)

data_X = df_fill.loc[:, df_fill.columns != "Outcome"]
data_Y = df_fill[["Outcome"]]

train_X, test_X, train_Y, test_Y = train_test_split(data_X, data_Y,
                                                    test_size=0.2,
                                                    stratify=data_Y,
                                                    random_state=0)

train_X.reset_index(drop=True, inplace=True)
test_X.reset_index(drop=True, inplace=True)
train_Y.reset_index(drop=True, inplace=True)
test_Y.reset_index(drop=True, inplace=True)

data_Y["Outcome"].value_counts()

feature_names = train_X.columns

scaler = StandardScaler()

## Fit to train_X
scaler.fit(train_X)

## transform train_X
train_X = scaler.transform(train_X)
train_X = pd.DataFrame(train_X, columns = feature_names)

## transform test_X
test_X = scaler.transform(test_X)
test_X = pd.DataFrame(test_X, columns = feature_names)



In [ ]:
class SimulatedAnnealing:

    def __init__(self, initial_temp, final_temp, cooling_rate, max_iterations, kernel_mutation_prob, 
                 solver_set, activation_set, hidden_layer_sizes_set, alpha_set, learning_rate_init_set):
        """
            Initialize simulated annealing variables.
        """

        self.initial_temp = initial_temp
        self.final_temp = final_temp
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iterations
        self.kernel_mutation_prob = kernel_mutation_prob

        self.fitness_over_time = []
        self.temps = []
        self.genome_over_time = []
        self.best_fitness_overtime = {}

        ## Evolved Hyperparameters
        self.solver = solver_set
        self.activation = activation_set
        self.hidden_layer_sizes = hidden_layer_sizes_set
        self.alpha = alpha_set
        self.learning_rate_init = learning_rate_init_set

    def accept_solution(self, change_in_fitness, temperature):
        """
            Accept the change in fitnees or not.
            :param change_in_fitness: The change in fitness --> New fitness - current fitness
            :param temperature: The temperature in that iteration
            :return: True is the change is accepted else False
        """

        ## If the change in fitness is greater than 0, the change is always accepted
        if change_in_fitness > 0:
            return True
        else:
        ## If the change in fitness is less than cero, accept the change with some probability
            p = np.exp(change_in_fitness / temperature)
            if np.random.rand() < p:
                ## Accept the change - return True
                return True
            else:
                ## Reject the change - return False
                return False

    def create_random_hyperparameters_set(self, my_solver, my_activation, my_hidden_layer_sizes, my_alpha, my_learing_rate_init):
        """
            Create random hyperparameters set for MLP model. Uses the previously defined possible parameters to randomly create the first genotype.
            :param solver: List with possible solvers
            :param activation: List with possible activation functions
            :param hidden_layer_sizes: Range of possible hidden layer sizes
            :param alpha: Range of possible alpha values
            :param learning_rate_init: Range of possible learning rate values

            :return: List with random MLP hyperparameter values
        """

        ## Create the output list
        hyper_params = [random.choice(my_solver), random.choice(my_activation), random.choice(my_hidden_layer_sizes), random.choice(my_alpha), random.choice(my_learing_rate_init)]

        return hyper_params

    def get_mlp_fitness(self, my_solver, my_activation, my_hidden_layer_sizes, my_alpha, my_learing_rate_init):
        """
            Cretes MLP model, train and test the model to get the model fitness
            :param solver: Selected MLP solver
            :param activation: Selected activation function
            :param hidden_layer_sizes: Selected hidden layer sizes value
            :param alpha: Selected alpha value
            :param learning_rate_init: Selected learning rate

            :return: MLP F1 Score
        """

        my_mlp = MLPClassifier(solver=my_solver, activation=my_activation, hidden_layer_sizes=my_hidden_layer_sizes, alpha=my_alpha, learning_rate_init=my_learing_rate_init, max_iter=10000)
        my_mlp.fit(train_X, train_Y.values.ravel())
        my_prediction = my_mlp.predict(test_X)
        my_f1_score = f1_score(test_Y, my_prediction, average='micro')

        return my_f1_score

    def random_number_close_range(self, current_value, x, int_value = False):
        """
            Take the current value, add and subtract a constant value x to generate a randon
            value in the range(current_value - x, current_value + x)
            :param current_value: Current value that is the set to be the main value in the range
            :param x: Value to add and subtract to current_value

            :return: Random number in range(current_value - x, current_value + x)
        """

        lower_bound = current_value - x
        upper_bound = current_value + x

        if lower_bound < 0:
            lower_bound = 0

        if int_value:
            ## Generate random integer number between lower and upper bounds
            random_rational_number = random.randint(lower_bound, upper_bound)

        else:
            ## Generate random rational number between lower and upper bounds
            random_rational_number = random.uniform(lower_bound, upper_bound)

        return random_rational_number

    def optimization(self):
        """
            Simulated annealing process.
        """

        ## Create a current random solution
        current_solution = self.create_random_hyperparameters_set(self.solver, self.activation, self.hidden_layer_sizes, self.alpha, self.learning_rate_init)

        ## Current fitness
        current_fitness = self.get_mlp_fitness(my_solver=current_solution[0], my_activation=current_solution[1], my_hidden_layer_sizes=current_solution[2], my_alpha=current_solution[3], my_learing_rate_init=current_solution[4])

        n_genes = len(current_solution)
        self.mutation_rate = 1/n_genes

        ## Set the initial temperature
        temperature = self.initial_temp

        ## We create a fitness_over_time that store all the fitness calculated, to keep track in the changes
        ## A copy is used, as if is not use the case where the change is not accepted, is going to change the 
        ## current_fitness, but using .copy() does not change the original value of current_fitness
        self.fitness_over_time.append(current_fitness.copy())
        ## Store the temperature through iterations
        self.temps.append(temperature)
        ## Store the genotype through iterations
        self.genome_over_time.append(current_solution.copy())

        ###########################################################

        ## Record start time
        start = time.time()

        ## Do it until reach the maximum iterations
        for iteration in range(self.max_iterations):

            # Generate a neighboring solution with mutation (gene flipping)
            new_solution = current_solution.copy()

            ## Calculate the probability of mutation for each gene and modify it if the random
            ## number generated is lower than the mutation_rate.
            ## This one could flip more than one gene, althought is more likely to not change 
            ## any gen due to the low probability
            for gene_no in range(len(new_solution)):
                
                if np.random.rand() < self.mutation_rate:

                    if gene_no == 0 and np.random.rand() < self.kernel_mutation_prob:
                        new_solution[gene_no] = random.choice(self.solver)
                    elif gene_no == 1 and np.random.rand() < self.kernel_mutation_prob:
                        new_solution[gene_no] = random.choice(self.activation)
                    elif gene_no == 2:
                        # new_solution[gene_no] = random.choice(self.hidden_layer_sizes)
                        new_solution[gene_no] = self.random_number_close_range(new_solution[gene_no], 2, int_value=True)
                    elif gene_no == 3:
                        # new_solution[gene_no] = random.choice(self.alpha)
                        new_solution[gene_no] = self.random_number_close_range(new_solution[gene_no], 0.01)
                    elif gene_no == 4:
                        # new_solution[gene_no] = random.choice(self.learning_rate_init)
                        new_solution[gene_no] = self.random_number_close_range(new_solution[gene_no], 0.001)

            ## Calculate the fitness for the new solution
            new_fitness = self.get_mlp_fitness(my_solver=new_solution[0], my_activation=new_solution[1], my_hidden_layer_sizes=new_solution[2], my_alpha=new_solution[3], my_learing_rate_init=new_solution[4])
            print(f"new_fitness: {new_fitness}")

            ## Calculate the change in the fitness, use to know whether to accepted or not later
            change_in_fitness = new_fitness - current_fitness
            print(f"change_in_fitness: {change_in_fitness}")

            ## Check if new solution should be accepted, if not accepted just go to the next iteration
            ## and ignore the new fitness
            if self.accept_solution(change_in_fitness, temperature):
                current_solution = new_solution.copy()
                current_fitness = new_fitness.copy()
            
            print(f"Current solution: {current_solution}")

            ## Add the new fitness to fitness_over_time to plot the changes in fitness later
            self.fitness_over_time.append(current_fitness.copy())
            print(f"fitness_over_time: {self.fitness_over_time}")

            ## Track solutions with high fitness
            print(f"Current fitness: {current_fitness}")
            if current_fitness >= 0.82:
                self.best_fitness_overtime[current_fitness] = current_solution

            ## Cool down
            temperature *= self.cooling_rate
            self.temps.append(temperature)
            print(f"temps: {self.temps}")

            print(f"iteration: {iteration}")

            print("------------------------------------------------------")

        ## Record en time
        end = time.time()

        ## Print execution time
        execution_time = end - start
        print(f"Execution time: {execution_time:.2f}")

        ##Print the final solution
        print(f"Best solution: {current_solution}")
        best_solution_fitness = current_fitness
        print(f"Best solution fitness: {best_solution_fitness}")

        ## Prin best fitness over time dict
        print(self.best_fitness_overtime)

        plt.plot(self.fitness_over_time)
        plt.title("Best Fitness Over Time")
        plt.xlabel("Iteration")
        plt.ylabel("Fitness")
        plt.show()

def main():
    ## Initial Simulated annealing values
    init_initial_temp = 100
    init_final_temp = 0.01
    init_cooling_rate = 0.99
    init_max_iterations = 1000
    init_kernel_mutation_prob = 0.03

    ## Initial hyperparameters
    init_solver = ['lbfgs', 'sgd', 'adam']
    init_activation = ['identity', 'logistic', 'tanh', 'relu']
    init_hidden_layer_sizes = np.arange(20, 200, 1)
    init_alpha = np.arange(0.0001, 0.1, 0.001)
    init_learning_rate_init = np.arange(0.001, 0.1, 0.001)

    ## cooling rate 0.99
    sa_optimization = SimulatedAnnealing(initial_temp=init_initial_temp, 
                                        final_temp=init_final_temp, 
                                        cooling_rate=init_cooling_rate, 
                                        max_iterations=init_max_iterations, 
                                        kernel_mutation_prob=init_kernel_mutation_prob,
                                        solver_set=init_solver,
                                        activation_set=init_activation,
                                        hidden_layer_sizes_set=init_hidden_layer_sizes,
                                        alpha_set=init_alpha,
                                        learning_rate_init_set=init_learning_rate_init)

    sa_optimization.optimization()

if __name__ == "__main__":
    main()